Milestone2


In [26]:
import kagglehub
from kagglehub import KaggleDatasetAdapter
import pandas as pd
import numpy as np

#kagglehub.login()


In [27]:
# !kaggle datasets download -d tonygordonjr/football-match-statistics-and-more
# !unzip /content/football-match-statistics-and-more.zip -d ./

In [28]:
teams_clean_col = ['fixture_id', 'team_id', 'team_name', 'fouls','yellow_cards', 'red_cards']
stats_clean_col = ['fixture_id', 'fixture_date', 'fixture_referee', 'league_id', 'league_name', 'teams_home_id', 'teams_home_name', 'teams_away_id', 'teams_away_name']
players_clean_col = ['fixture_id', 'team_id', 'team_name', 'player_id', 'player_name', 'fouls_committed', 'yellow_cards', 'red_cards', 'game_minutes']

fix_teams_clean = pd.read_csv("fixture_stats_teams_clean.csv",usecols=teams_clean_col)
fix_clean = pd.read_csv("fixture_stats_clean.csv", usecols=stats_clean_col)
fix_players_clean = pd.read_csv("fixture_stats_players_clean.csv",usecols=players_clean_col)

In [29]:
fix_players_clean.drop(fix_players_clean[fix_players_clean["game_minutes"].isna()].index, inplace=True)

In [30]:
fix_players_clean["game_minutes"].isna().sum()

0

In [31]:
fix_players_clean["game_minutes"].sum()

64253242.0

In [32]:
import pandas as pd


# Clean all data before 2020. jan. 1.
fix_clean["fixture_date"] = pd.to_datetime(fix_clean["fixture_date"]).dt.tz_localize(None)
start_date = pd.Timestamp("2020-01-01")
fix_clean = fix_clean[fix_clean["fixture_date"] >= start_date]


print(fix_clean.head())



    fixture_id        fixture_date       fixture_referee  league_id  \
0      1299262 2025-01-30 20:00:00     Match Not Started          3   
1       566789 2020-07-06 18:30:00  Felix Brych, Germany         78   
2       566788 2020-07-02 18:30:00             F. Zwayer         78   
28     1119329 2023-08-30 19:00:00             I. Kovacs          2   
29      946878 2022-09-14 19:00:00             I. Peljto          2   

              league_name  teams_home_id   teams_home_name  teams_away_id  \
0      UEFA Europa League            604  Maccabi Tel Aviv            212   
1              Bundesliga            180     FC Heidenheim            162   
2              Bundesliga            162     Werder Bremen            180   
28  UEFA Champions League            400     FC Copenhagen           3491   
29  UEFA Champions League            400     FC Copenhagen            536   

      teams_away_name  
0            FC Porto  
1       Werder Bremen  
2       FC Heidenheim  
28  Raków Częs

In [33]:

count3 = fix_clean.shape[0]

# Keep only the 4 main leagues
top_leagues = ["La Liga", "Serie A", "Bundesliga", "Premier League"]
fix_clean = fix_clean[fix_clean["league_name"].isin(top_leagues)]

count4 = fix_clean.shape[0]
print(f"Number of rows before filtering: {count3}")
print(f"Number of rows after filtering: {count4}")

Number of rows before filtering: 19135
Number of rows after filtering: 7484


In [34]:
count1 = fix_teams_clean.shape[0]

# Drop all unneccessary teams' datas
team_ids_to_keep = fix_players_clean["team_id"].unique()
fix_teams_clean = fix_teams_clean[fix_teams_clean["team_id"].isin(team_ids_to_keep)]
count2 = fix_teams_clean.shape[0]

print(f"Number of rows before filtering: {count1}")
print(f"Number of rows after filtering: {count2}")

Number of rows before filtering: 65220
Number of rows after filtering: 64693


In [35]:
print(fix_players_clean.columns)
print(fix_teams_clean.columns)
print(fix_clean.columns)

Index(['fixture_id', 'team_id', 'team_name', 'player_id', 'player_name',
       'game_minutes', 'fouls_committed', 'yellow_cards', 'red_cards'],
      dtype='object')
Index(['fixture_id', 'team_id', 'team_name', 'fouls', 'yellow_cards',
       'red_cards'],
      dtype='object')
Index(['fixture_id', 'fixture_date', 'fixture_referee', 'league_id',
       'league_name', 'teams_home_id', 'teams_home_name', 'teams_away_id',
       'teams_away_name'],
      dtype='object')


In [36]:

merged = fix_clean.merge(fix_teams_clean, on="fixture_id", how="inner")
merged = merged[merged["team_id"] == merged["teams_home_id"]]
merged = merged.rename(columns={"fouls": "home_team_fouls", "yellow_cards": "home_team_yellow_cards", "red_cards": "home_team_red_cards"})
merged = merged.drop(columns=["team_id", "team_name"])

merged = merged.merge(fix_teams_clean, on="fixture_id", how="inner")
merged = merged[merged["team_id"] == merged["teams_away_id"]]
merged = merged.rename(columns={"fouls": "away_team_fouls", "yellow_cards": "away_team_yellow_cards", "red_cards": "away_team_red_cards"})
merged = merged.drop(columns=["team_id", "team_name"])

merged["total_cards"] = merged["home_team_yellow_cards"] + merged["away_team_yellow_cards"] + merged["home_team_red_cards"] * 2 + merged["away_team_red_cards"] * 2
merged
fix_with_cards = merged


In [37]:
pd.set_option('display.max_columns', 100)
merged[merged['fixture_id'] == 1208526]

,fixture_id,fixture_date,fixture_referee,league_id,league_name,teams_home_id,teams_home_name,teams_away_id,teams_away_name,home_team_fouls,home_team_yellow_cards,home_team_red_cards,away_team_fouls,away_team_yellow_cards,away_team_red_cards,total_cards
7387,1208526,2024-08-31 19:30:00,Mario Melero,140,La Liga,537,Leganes,798,Mallorca,11,1,0,17,3,0,4


In [38]:
# Needed in the final table:
#fixture referee, league_name, teams_home_name, teams_away_name, avg_home_team_fouls, avg_home_team_yellow_cards, avg_home_team_red_cards, avg_away_team_fouls, avg_away_team_yellow_cards, avg_away_team_red_cards, [avg_total_cards_player_{1-22}, avg_total_fouls_player_{1_22},] total_cards(yellow_cards+red_cards*2)


In [39]:
# TODO 1 - DONE
# Meg kéne csinálni az átlagokat home és away csapatra is szabálytalanságokból, sárga és piros lapokból
# LEHET EGYSZERŰBB EGYBŐL TODO 2-VEL PRÓBÁLKOZNI!

merged['avg_home_team_fouls'] = merged.groupby('teams_home_name')['home_team_fouls'].transform('mean')
merged['avg_home_team_yellow_cards'] = merged.groupby('teams_home_name')['home_team_yellow_cards'].transform('mean')
merged['avg_home_team_red_cards'] = merged.groupby('teams_home_name')['home_team_red_cards'].transform('mean')
merged['avg_away_team_fouls'] = merged.groupby('teams_away_name')['away_team_fouls'].transform('mean')
merged['avg_away_team_yellow_cards'] = merged.groupby('teams_away_name')['away_team_yellow_cards'].transform('mean')
merged['avg_away_team_red_cards'] = merged.groupby('teams_away_name')['away_team_red_cards'].transform('mean')

In [40]:
merged

,fixture_id,fixture_date,fixture_referee,league_id,league_name,teams_home_id,teams_home_name,teams_away_id,teams_away_name,home_team_fouls,home_team_yellow_cards,home_team_red_cards,away_team_fouls,away_team_yellow_cards,away_team_red_cards,total_cards,avg_home_team_fouls,avg_home_team_yellow_cards,avg_home_team_red_cards,avg_away_team_fouls,avg_away_team_yellow_cards,avg_away_team_red_cards
0,566789,2020-07-06 18:30:00,"Felix Brych, Germany",78,Bundesliga,180,FC Heidenheim,162,Werder Bremen,16,2,0,7,2,0,4,12.444444,1.722222,0.000000,12.194030,2.313433,0.119403
3,566788,2020-07-02 18:30:00,F. Zwayer,78,Bundesliga,162,Werder Bremen,180,FC Heidenheim,7,4,1,16,2,0,8,11.701493,1.970149,0.044776,13.888889,1.500000,0.055556
5,868032,2022-10-01 14:00:00,A. Madley,39,Premier League,40,Liverpool,51,Brighton,9,1,0,13,1,0,2,10.413043,1.250000,0.021739,10.387097,1.795699,0.096774
7,592286,2020-12-27 16:30:00,K. Friend,39,Premier League,40,Liverpool,60,West Brom,8,0,0,5,1,0,1,10.413043,1.250000,0.021739,10.473684,1.315789,0.105263
8,1035339,2023-12-17 16:30:00,M. Oliver,39,Premier League,40,Liverpool,33,Manchester United,13,2,0,8,6,1,10,10.413043,1.250000,0.021739,11.695652,2.195652,0.021739
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14214,878055,2022-10-29 12:00:00,Carlos del Cerro,140,La Liga,723,Almeria,538,Celta Vigo,10,4,0,7,0,1,6,12.473684,2.447368,0.131579,13.193548,2.430108,0.118280
14216,878215,2023-04-09 16:30:00,José Sánchez,140,La Liga,723,Almeria,532,Valencia,11,3,0,11,2,0,5,12.473684,2.447368,0.131579,13.193548,2.365591,0.204301
14218,877967,2022-08-27 20:00:00,Jesús Gil,140,La Liga,723,Almeria,536,Sevilla,18,5,0,11,8,0,13,12.473684,2.447368,0.131579,12.150538,2.634409,0.139785
14220,878075,2022-11-09 18:00:00,Alejandro Muñiz,140,La Liga,723,Almeria,546,Getafe,17,8,1,8,4,0,14,12.473684,2.447368,0.131579,16.806452,3.473118,0.182796


In [43]:
# TODO 2 - DONE
# Az átlagokat úgy kéne megcsinálni hogy csak az azon mecs előtti infóknak legyen az átlaga
merged = merged.sort_values(by=["teams_home_id", "fixture_date"])
merged["avg_home_team_fouls"] = merged.groupby("teams_home_id")["home_team_fouls"] \
.expanding().mean().shift(1).reset_index(level=0, drop=True).fillna(0)
merged


,fixture_id,fixture_date,fixture_referee,league_id,league_name,teams_home_id,teams_home_name,teams_away_id,teams_away_name,home_team_fouls,home_team_yellow_cards,home_team_red_cards,away_team_fouls,away_team_yellow_cards,away_team_red_cards,total_cards,avg_home_team_fouls,avg_home_team_yellow_cards,avg_home_team_red_cards,avg_away_team_fouls,avg_away_team_yellow_cards,avg_away_team_red_cards
8832,157231,2020-01-11 15:00:00,"Anthony Taylor, England",39,Premier League,33,Manchester United,71,Norwich,18,0,0,7,0,0,0,0.000000,1.827957,0.053763,9.000000,1.571429,0.035714
4203,157251,2020-01-22 20:15:00,"Jonathan Moss, England",39,Premier League,33,Manchester United,44,Burnley,11,0,0,7,2,0,2,18.000000,1.827957,0.053763,10.575758,1.863636,0.121212
453,157260,2020-02-01 17:30:00,"Paul Tierney, England",39,Premier League,33,Manchester United,39,Wolves,14,3,0,15,2,0,5,14.500000,1.827957,0.053763,11.473118,1.870968,0.053763
8816,157281,2020-02-23 14:00:00,"Martin Atkinson, England",39,Premier League,33,Manchester United,38,Watford,6,0,0,15,1,0,1,14.333333,1.827957,0.053763,12.500000,1.607143,0.071429
8882,157301,2020-03-08 16:30:00,"Mike Dean, England",39,Premier League,33,Manchester United,50,Manchester City,11,2,0,9,4,0,6,12.250000,1.827957,0.053763,8.591398,1.473118,0.053763
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1304,1223662,2024-10-06 16:00:00,F. La Penna,135,Serie A,1579,Monza,497,AS Roma,16,3,0,13,1,0,4,11.756098,2.304348,0.065217,12.913978,2.516129,0.096774
1312,1223680,2024-10-27 13:00:00,A. Rapuano,135,Serie A,1579,Monza,517,Venezia,17,4,1,16,2,0,8,11.857143,2.304348,0.065217,13.703704,2.777778,0.148148
5261,1223700,2024-11-02 19:45:00,E. Feliciani,135,Serie A,1579,Monza,489,AC Milan,18,2,0,15,1,0,3,11.976744,2.304348,0.065217,12.247312,2.333333,0.139785
5265,1223712,2024-11-10 17:00:00,A. Colombo,135,Serie A,1579,Monza,487,Lazio,13,5,0,10,1,0,6,12.113636,2.304348,0.065217,12.052632,2.631579,0.094737


In [42]:
# TODO 3 (nem prioritás)
# Ugyanígy kéne az átlagos szabálytalanságot és lapszámot kiszámolni mind a 22 játékosra az adott meccsen (3*22 = 66 új oszlop)
#
# Valid ha csak megcsináljuk az első két TODO-t és megkérdezzük, hogy a játékosok tekintetében jó lesz-e így